In [1]:
from pathlib import Path
from sys import stdout
import numpy as np
import pandas as pd

from pymatgen.transformations.standard_transformations import SupercellTransformation

from neighbormodels.structure import from_file
from neighbormodels.neighbors import count_neighbors
from neighbormodels.interactions import build_model

import os

In [10]:
fcc_filepath = "data/a1-fcc.cif"
fe_structure_fcc = from_file(structure_file=fcc_filepath)
fe_structure_fcc[:] ="Fe" 
print(fe_structure_fcc)

Full Formula (Fe4)
Reduced Formula: Fe
abc   :   3.614910   3.614910   3.614910
angles:  90.000000  90.000000  90.000000
Sites (4)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Fe    0    0    0
  1  Fe    0    0.5  0.5
  2  Fe    0.5  0    0.5
  3  Fe    0.5  0.5  0


In [17]:
N = 5
r = 4
fe_structure_supercell = fe_structure_fcc.copy()
fe_structure_supercell.make_supercell([N,N,N])
fe_neighbor_data = count_neighbors(fe_structure_supercell, r)
a = fe_neighbor_data.data_frame \
    .merge(fe_neighbor_data.data_frame.rename(columns={"j": "i", "i": "j"}), how="outer") \
    .sort_values(["i","distance_bin", "j"]) \
    .loc[:, ["i", "j", "distance_bin"]] \
    .reset_index(drop=True)

neighbors_per_site = a.groupby(["i"]).count().reset_index().rename(columns={"j": "count"}).loc[:, "count"].values
sites_per_distance_group = a.groupby(["i", "distance_bin"]).count().reset_index() \
                            .rename(columns={"j": "count"}).loc[:, "count"].values
neighbor_indices = a["j"].values


neighbor_count = np.dstack ( np.unique(sites_per_distance_group, return_index=True) )    
neighbor_count.dtype = np.dtype([('v', neighbor_count.dtype), ('i', neighbor_count.dtype)])
neighbor_count.sort(order='i', axis=1)
neighbor_count = neighbor_count.flatten()['v'].tolist()

print("Number of sites:", 4*N*N*N)
print("Number of 1st and 2nd neighbors for each index i:", neighbor_count)
print("Neighbor indices sorted over index i and distances: ", neighbor_indices)

Number of sites: 500
Number of 1st and 2nd neighbors for each index i: [12, 6]
Neighbor indices sorted over index i and distances:  [125 129 145 ... 494 495 498]


In [18]:
filename = os.getcwd()+'/neighbor_lists.txt'

n_sites = np.array([4*N*N*N])
f=open(filename,'w')
f.write("\n".join(" ".join(map(str, x)) for x in (n_sites,neighbor_count)))
np.savetxt(filename, neighbor_indices,fmt='%d')
f.close()